# Conjunto de dados para previsão de gordura corporal
##### Estimativas de gordura corporal e diversas medidas de circunferência corporal para 252 homens.

# Predição de Gordura Corporal com CRISP-DM

Este notebook implementa uma análise exploratória e um modelo de regressão para prever o percentual de gordura corporal (body fat) com base em variáveis antropométricas. Todo o processo segue a metodologia CRISP-DM (Cross-Industry Standard Process for Data Mining), abrangendo desde o entendimento do negócio até a modelagem e avaliação.

## 1. Entendimento do Negócio (Business Understanding)

**Objetivo:**  
Desenvolver um modelo de Machine Learning capaz de prever o percentual de gordura corporal a partir de medidas físicas como peso, altura, circunferências corporais e dobras cutâneas. Esse tipo de estimativa pode ser útil em academia, saúde preventiva, nutrição e avaliação física.

**Motivação:**  
Medições diretas de gordura corporal, como DEXA ou pesagem hidrostática, são caras e pouco acessíveis. Um modelo preditivo baseado em medidas simples e de baixo custo pode democratizar esse tipo de avaliação.

**Métricas de Sucesso:**  
Como se trata de um problema de regressão, serão priorizadas métricas como:

- **MAE** (Mean Absolute Error)  
- **RMSE** (Root Mean Squared Error)  
- **R²** (Coeficiente de Determinação)

Um modelo de sucesso deve prever a gordura corporal com erro médio suficientemente baixo para ser aplicável em avaliações físicas reais.

## 2. Entendimento dos Dados (Data Understanding)

Nesta etapa, exploramos o dataset para compreender sua estrutura, distribuição das variáveis, possíveis inconsistências e relacionamentos importantes para a modelagem.

### 2.1 Carregamento e Visão Geral Inicial

O dataset **[Body Fat Prediction Dataset](https://www.kaggle.com/datasets/fedesoriano/body-fat-prediction-dataset)**, disponível no Kaggle, contém medições antropométricas de indivíduos, incluindo:

- Idade  
- Peso  
- Altura  
- Circunferências corporais (abdômen, quadril, peito, coxa etc.)  
- Dobras cutâneas  
- Percentual de gordura corporal (variável alvo)

Nesta fase investigaremos:

- Estatísticas descritivas  
- Distribuição das variáveis  
- Outliers  
- Correlações entre atributos  
- Possíveis problemas de qualidade ou inconsistência nos dados  

Esses insights orientarão as etapas seguintes de preparação e modelagem.

In [2]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd

# Nome do arquivo principal do dataset no Kaggle
file_path = "bodyfat.csv"

# Carregar o dataset usando KaggleHub
df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "fedesoriano/body-fat-prediction-dataset",
    file_path,
)

# Visão geral inicial
print("Formato do dataset:", df.shape)
display(df.head())

C:\Users\elias\AppData\Local\Temp\ipykernel_9504\210618376.py:9: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


Formato do dataset: (252, 15)


,Density,BodyFat,Age,Weight,Height,Neck,Chest,Abdomen,Hip,Thigh,Knee,Ankle,Biceps,Forearm,Wrist
0,1.0708,12.3,23,154.25,67.75,36.2,93.1,85.2,94.5,59.0,37.3,21.9,32.0,27.4,17.1
1,1.0853,6.1,22,173.25,72.25,38.5,93.6,83.0,98.7,58.7,37.3,23.4,30.5,28.9,18.2
2,1.0414,25.3,22,154.00,66.25,34.0,95.8,87.9,99.2,59.6,38.9,24.0,28.8,25.2,16.6
3,1.0751,10.4,26,184.75,72.25,37.4,101.8,86.4,101.2,60.1,37.3,22.8,32.4,29.4,18.2
4,1.0340,28.7,24,184.25,71.25,34.4,97.3,100.0,101.9,63.2,42.2,24.0,32.2,27.7,17.7


In [3]:
print("\nTipos das variáveis:")
print(df.dtypes)


Tipos das variáveis:
Density    float64
BodyFat    float64
Age          int64
Weight     float64
Height     float64
Neck       float64
Chest      float64
Abdomen    float64
Hip        float64
Thigh      float64
Knee       float64
Ankle      float64
Biceps     float64
Forearm    float64
Wrist      float64
dtype: object


In [4]:
print("\nEstatísticas descritivas:")
display(df.describe())


Estatísticas descritivas:


,Density,BodyFat,Age,Weight,Height,Neck,Chest,Abdomen,Hip,Thigh,Knee,Ankle,Biceps,Forearm,Wrist
count,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000
mean,1.055574,19.150794,44.884921,178.924405,70.148810,37.992063,100.824206,92.555952,99.904762,59.405952,38.590476,23.102381,32.273413,28.663889,18.229762
std,0.019031,8.368740,12.602040,29.389160,3.662856,2.430913,8.430476,10.783077,7.164058,5.249952,2.411805,1.694893,3.021274,2.020691,0.933585
min,0.995000,0.000000,22.000000,118.500000,29.500000,31.100000,79.300000,69.400000,85.000000,47.200000,33.000000,19.100000,24.800000,21.000000,15.800000
25%,1.041400,12.475000,35.750000,159.000000,68.250000,36.400000,94.350000,84.575000,95.500000,56.000000,36.975000,22.000000,30.200000,27.300000,17.600000
50%,1.054900,19.200000,43.000000,176.500000,70.000000,38.000000,99.650000,90.950000,99.300000,59.000000,38.500000,22.800000,32.050000,28.700000,18.300000
75%,1.070400,25.300000,54.000000,197.000000,72.250000,39.425000,105.375000,99.325000,103.525000,62.350000,39.925000,24.000000,34.325000,30.000000,18.800000
max,1.108900,47.500000,81.000000,363.150000,77.750000,51.200000,136.200000,148.100000,147.700000,87.300000,49.100000,33.900000,45.000000,34.900000,21.400000


In [5]:
print("\nValores ausentes por coluna:")
print(df.isnull().sum())


Valores ausentes por coluna:
Density    0
BodyFat    0
Age        0
Weight     0
Height     0
Neck       0
Chest      0
Abdomen    0
Hip        0
Thigh      0
Knee       0
Ankle      0
Biceps     0
Forearm    0
Wrist      0
dtype: int64
